In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string

from tensorflow.keras import layers
from tensorflow.keras import losses

# requires update to tensorflow 2.4
# >>> conda activate PIC16B
# >>> pip install tensorflow==2.4
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers.experimental.preprocessing import StringLookup

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Import Data

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/benbrill/MoodSpace/main/data/trainingSongs_clean.csv")
df.head()

,Unnamed: 0,trackName,artist,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,lyrics,cluster,language
0,3,Overthinker,INZO,4K9xid96G3YmIvQZXN9SXg,0.472,0.605,8.0,-4.437,1.0,0.1340,0.03110,0.030800,0.1010,0.212,128.375,audio_features,a person who thinks all the time has nothing t...,0,en
1,4,Lifestyles of the Rich & Famous,Good Charlotte,2g2a5kDeZexbUTD8abcvm6,0.620,0.930,1.0,-3.685,1.0,0.0374,0.00043,0.000000,0.0686,0.609,106.220,audio_features,always see it on t v or read in the magazines ...,3,en
2,6,Carrying Your Love With Me,George Strait,7puxIVNdj5nsBJk43zM3bH,0.629,0.479,10.0,-10.608,1.0,0.0271,0.22000,0.000000,0.0587,0.345,138.231,audio_features,baby all i got is this beat up leather bag and...,5,en
3,7,"Check Yes, Juliet",We The Kings,0wVluBsVAVzBKrqspuCcwR,0.352,0.912,7.0,-4.253,1.0,0.0725,0.00197,0.000000,0.1930,0.351,166.795,audio_features,check yes juliet are you with me rain is falli...,2,en
4,8,At My Worst (feat. Kehlani),Pink Sweat$,58w68w4s8h9gw3xrDaXyuj,0.731,0.484,0.0,-5.579,1.0,0.0354,0.73000,0.000003,0.3260,0.350,92.043,audio_features,can i call you baby can you be my friend can y...,1,en


# Create Text Vectorization

In [3]:
max_tokens = 200
sequence_length = 500

vectorize_layer = TextVectorization(
    max_tokens=max_tokens, # only consider this many words
    output_mode='int',
    output_sequence_length=sequence_length) 

In [4]:
data = tf.data.Dataset.from_tensor_slices((df["lyrics"], df["cluster"]))

In [7]:
data = data.shuffle(buffer_size = len(data))

train_size = int(0.7*len(data))
val_size   = int(0.1*len(data))

vectorize_layer.adapt(data)

train = data.take(train_size)
val   = data.skip(train_size).take(val_size)
test  = data.skip(train_size + val_size)
len(train), len(val), len(test)

ValueError: The dataset passed to 'adapt' must contain a single tensor value.

In [6]:
def vectorize_movie_script(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), [label]

train_vec = train.map(vectorize_movie_script)
val_vec   = val.map(vectorize_movie_script)
test_vec  = test.map(vectorize_movie_script)

# Create Model

In [13]:
model = tf.keras.Sequential([
  layers.Embedding(max_tokens, output_dim = 10, name="embedding"),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(8)]
)

In [14]:
model.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam', 
              metrics=['accuracy'])

In [15]:
history = model.fit(train_vec, epochs = 100, validation_data = val_vec)

Epoch 1/100
501/501 [==============================] - 3s 3ms/step - loss: 2.0551 - accuracy: 0.1527 - val_loss: 1.9799 - val_accuracy: 0.1408
Epoch 2/100
501/501 [==============================] - 1s 3ms/step - loss: 1.9672 - accuracy: 0.1771 - val_loss: 1.9176 - val_accuracy: 0.2817
Epoch 3/100
501/501 [==============================] - 1s 3ms/step - loss: 1.9345 - accuracy: 0.1830 - val_loss: 1.8964 - val_accuracy: 0.2113
Epoch 4/100
501/501 [==============================] - 1s 2ms/step - loss: 1.8922 - accuracy: 0.2253 - val_loss: 1.8152 - val_accuracy: 0.3239
Epoch 5/100
501/501 [==============================] - 1s 2ms/step - loss: 1.8866 - accuracy: 0.2111 - val_loss: 1.8606 - val_accuracy: 0.2394
Epoch 6/100
501/501 [==============================] - 1s 3ms/step - loss: 1.8955 - accuracy: 0.1835 - val_loss: 1.9304 - val_accuracy: 0.2535
Epoch 7/100
501/501 [==============================] - 1s 2ms/step - loss: 1.8831 - accuracy: 0.2061 - val_loss: 1.8494 - val_accuracy: 0.2535

In [16]:
model.predict(test_vec)[1]

array([ 4.5185795e-01,  4.9152628e-01, -2.3496413e-01, -6.6743582e-02,
       -2.0310804e-03,  3.1183892e-01, -9.1974133e-01, -4.3409243e+00],
      dtype=float32)

In [19]:
model.evaluate(test_vec)

145/145 [==============================] - 0s 928us/step - loss: 1.8582 - accuracy: 0.2483


[1.8582499027252197, 0.24827586114406586]